In [1]:
#Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pulp

from sklearn.preprocessing import MinMaxScaler

# Abstract

In this post, we present an implementation for *Next-Generation Optimization for Dasher Dispatch at DoorDash* [@doordash_optimization_2020] and *Using ML and Optimization to Solve DoorDash’s Dispatch Problem* [@weinstein_ml_optimization_2021]. We demonstrate this implementation in the context of a smaller, simpler food delivery system. We conclude by presenting some simple sensitivity analysis from changes in our hyperparameters weights in our scoring methods. The code for this simulation can be found at the [github repo](https://github.com/thebayesianbandit/thebayesianbandit.github.io/blob/main/posts/dispatch_alg/dispatch_alg.ipynb).

# Introduction

**Food-Ez** is a new food delivery service specializing in delivering food in small, walkable cities. Like other companies like DoorDash and UberEats, Food-Ez is a three sided marketplace, dealing with the needs and wants of customers (those who place the orders), restaurants (those who create the orders), and deliverers (those who fulfill the orders). Understanding this, Food-Ez developed a novel dispatch algorithm to maximize efficiency in their marketplace. Specifically, they created an algorithm that minimized delivery time by matching orders to drivers closest to the restaurant. While this approach has been effective for Food-Ez, they are looking to upgrade their current algorithm in a few ways. First, they are **allowing deliverers to be assigned up to 2 orders at a time**. Second, they want to have more efficient operations. Their current algorithm has drivers waiting quite a bit of time for food to be prepped at restaurants. They want to **maximize deliverer efficiency by minimizing driver wait and idle time**. 

# Designing the Problem via Control

Before going into the details of how each algorithm operates within this problem, we first present how to formulate the problem to achieve optimal solutions. One way to approach this problem is by viewing it as a **control problem**. Control theory offers a rich framework by which we can design our problem and solutions. In essence, control theory aims to stabilize/optimize some system that we'll denote as $P$ (commonly known as the Plant). The system creates outputs that include feedback signals that are fed back to a **controller** denoted as $C$. @fig-control-diag is an example control diagram.

![An example control diagram](http://www.polytechnichub.com/wp-content/uploads/2017/04/closed_loop.jpg){#fig-control-diag}

Control theory excels at helping us model dynamical systems and how we can control them even in the face of uncertainty (i.e. not knowing all the dynamics of the system). In our scenario specifically, the delivery marketplace is a highly dynamic system. Orders come in at varying times and locations with an unknown supply of drivers to deliver orders. With control theory, we can boil down this complex problem into essentially three parts: First, what is the system we are trying to control? Second, what information can we gain from the system? Third, what actions can we take to ensure the system operates accordingly?

## What's the system?

The first question can be answered by understanding the business objective. We want customers to utilize our service for their food delivery needs. We promote our service by promising fast and accurate delivery right to their doors. Additionally, since our drivers are critical to the operations of this business, we want to keep our drivers as busy as possible to ensure they maximize earnings. Essentially, our business objective is to maintain optimal customer service to those who order while also providing good opportunities to our deliverers to make money. How can we optimize for both objectives simultaneously? Or rather, what system are we trying to control to fulfill this objective? **The system we are aiming to control for Food-Ez is the queue**. If we can control the queue (i.e. drive the queue to zero), customers orders will be assigned quickly and be delivered quickly. Subsequently, if the queue is managed correctly, drivers' objective of maximizing earnings will also be optimized. The queue is the system for which we will be designing a controller.

## What should we feedback?

Now that we have identified what the system is, we can identify what elements of the system we should report back to the controller. If we are aiming to optimize the queue, we should report back metrics of queue health. Things like queue length, status of orders, status of drivers, location of dropoff and pickup, etc. The list below enumerates some of the crucial feedback elements we will provide to the controller. 

1. Order Status (Pending, Assigned, En Route, Delivered)
2. Driver Status (Idle, Assigned, En Route, Delivered)
3. Order Location (Pickup Location, Dropoff Location)
4. Driver Location 
5. Driver Assigned Orders

For question two, we are essentially defining **the states** of important elements in our system. In control theory, this fundamental approach is known as representing a system using state-space equations. The formal equations are found in @eq-cont-1.

$$
\dot{\mathbf{x}}(t) = \mathbf{A}\mathbf{x}(t) + \mathbf{B}\mathbf{u}(t)
$$
$$
\mathbf{y}(t) = \mathbf{C}\mathbf{x}(t) + \mathbf{D}\mathbf{u}(t)
$${#eq-cont-1}

@eq-cont-1 states that the rate of change of the state vector $\dot{x}(t)$ at time $t$ is equal to the current state vector $x(t)$ multiplied by the dynamics matrix $A$, plus the input vector $u(t)$ multiplied by the input matrix $B$. The system's output, $y(t)$, is equal to the current state vector of the system $x(t)$ multiplied by the output matrix $C$ plus input vector $u(t)$ multiplied by the feedthrough matrix $D$. Relating these equations back to our control diagram, the controller observes $y(t)$ and compares it against a predetermined reference or baseline. Based on that judgement, the controller sends inputs $u(t)$ back to the system. This input, along with the current state of the system, determines the instantaneous rate of change of the system's state. This state then produces output $y(t)$, and the process repeats itself. 

A classic example of this is cruise control in a car. The controller observes the car's speed and compares it against the set cruise speed. It then sends inputs to the engine to adjust the speed accordingly. These inputs, alongside other dynamics (e.g., drag, incline), influence the car's state, which in turn contributes to the measured output $y(t)$.

While these explanations have been incredibly simplified, they illustrate the important principle that a system can be adjusted in a closed-loop by providing crucial feedback to the controller on its current operation. In our scenario, we want to know the locations and destinations of drivers, the status of food preparation, and other relevant parameters, in order to optimally assign orders within the queue and achieve our business objectives.

## What actions can we take to adjust the system?

We identified the system and listed some information that would be useful for us to understand how the system is operating. We now need to implement actions that optimize the system's efficiency. In the previous section(s), we have identified the key objective. We need to assign orders to drivers such that orders get to their locations ASAP while also providing enough work to the drivers. There are several approaches to address this problem and each has its own advantages and disadvantages. We could implement logic such that we always assign the closest driver the order that comes in. Another approach would be holding orders for longer in the queue to see if a more "optimally" positioned driver comes along to take multiple orders nearby. We need sound strategy to identify which algorithmic process we should implement. For Food-Ez, it is optimizing the queue such that we minimize delivery time and maximize driver utilization (i.e. minimize driver idle time). Therefore, we need to design a controller that will balance these two objectives. 

# Establishing Operations

Before getting into the controller logic, we'll first walk through the operational flow of our system. This is illustrated in @fig-op-1 (which is provided by [@doordash_optimization_2020]).

![Flow of operations for delivery services like DoorDash and Food-Ez](https://doordash.engineering/wp-content/uploads/2020/08/life-cycle-12.jpg){#fig-op-1}

Walking through @fig-op-1, the process begins with a customer placing an order. The order is processed by the food delivery service and enters the queue. Returning to our control design, the queue sends pertinent information to the controller. The controller "analyzes" the information and sends its "suggested action" back to the queue. The queue then performs the actions by notifying orders to the restaurant (merchant) and sending orders to the "optimal" deliverer (dasher). The order is then picked up by the deliverer and drops off the order. 

Once again, we realize the importance of the "controller" in this system. The entirety of the delivery process is determined by the controller. In the next sections, we'll walk through the previous controller logic and the new proposed controller logic. 

## Current approach: bipartite matching

The original logic followed by Food-Ez (and additionally DoorDash) was designed by framing the task as a **bipartite matching problem**. Essentially, this problem involves finding a set of edges in a bipartite graph such that no two edges share a common vertex, thereby matching elements from two distinct sets. In the context of food delivery, this essentially means every order can only have one driver and every driver can only have one order. To do this efficiently to fulfill business objectives such as minimizing delivery time, problems like this can be solved using algorithms like the Hungarian algorithm [@kuhn1955hungarian]. 

While this approach fulfills the business objective, simple examples illustrate the weaknesses of this approach. Suppose a deliverer is assigned an order and is heading to the restaurant. On the way, the driver passes another restaurant that just received an order that will be ready shortly. Furthermore, the dropoff location of this order is nearby the dropoff location of the first order. Instead of assigning this order to the driver, the system must assign another driver to this order. While more drivers are utilized in this approach, drivers could be used more efficiently. 

## An example via queue theory

To illustrate this, let's pose the simple scenario of a single deliverer working for Food-Ez. Utilizing queue theory, we'll assume this is an M/M/1 queue where the arrival process is modeled by a Poisson distribution with parameter $\lambda = 1$ (per hour), the service distribution is an exponential distribution with $\mu = 2$ (per hour), and there is a single server (deliverer).

With these metrics, we can calculate system utilization (or driver utilization in our case). This is defined as $\rho = \frac{\lambda}{\mu}$, or the ratio between arrival rate and service rate. In our case, for a deliverer in the bipartite matching system, we get 50% driver utilization. While this is not bad, the average queue wait time for an order is defined as $W_{q} = \frac{L_{q}}{\lambda}$, where $L_{q}$ is average length of queue. The average length of the queue here is .5 so in our scenario, queue wait time is about .5 hours or 30 minutes. Since our deliverer can only handle one order at a time, every order on average would have to wait 30 minutes in the queue. 

In a different approach, let's say a deliverer can now take two orders concurrently and assume it is no further than a 10 minute drive to complete. In this case, a deliverer can fulfill 2 orders in about 40 minutes (1 order / 30 mins was the rate before). Converting this to hours, a deliverer can now fulfill 3 orders per hour, thus changing our deliverer utilization rate from 50% to 33%. More importantly, the queue wait time decreases from 30 minutes to 10 minutes (average queue length in this scenario is ~.167 orders, which is then divided by 1 order per hour). Not only could our drivers stay more busy if arrival rates increased, but our customers would have to wait only a third of the old time (if our simple assumptions hold)!

# Optimizing Dispatch using Integer Programming

So far we have demonstrated from a high-level the current operations of Food-Ez: a customer places an order, the controller (using a dispatch algorithm) matches orders to restaurants and deliverers to orders such that we minimize delivery time from order placement to order delivery. To accommodate other business objectives, we need a new optimization framework for our dispatch algorithm. Once implemented, we can substitute the current dispatch algorithm for the newly enhanced one and it should fit into our control design seamlessly. Our proposed solution for accomodating multiple business objectives is **integer programming with a weighted sum score** [@Gomory1958] (Note: [@doordash_optimization_2020] uses a mixed integer program approach due to the more complex and wholistic approach to addressing the dispatch problem, but for our simple scenario, we reduce the problem to an integer program). Our formulation is found in @eq-int-1.

$$
min \sum_{i=1}^{N}\sum_{j=1}^{D} c_{ij}x_{ij}
$$
$$
\sum_{j=1}^{D} x_{ij} = 1 \; \forall i \in N
$$
$$
\sum_{i=1}^{N} x_{ij} \le 2 \; \forall j \in D
$$
$$
c_{ij} = \sum_{m=1}^{W} w_{m}p_{m}
$${#eq-int-1}

Let $N$ be the set of unassigned orders and let $D$ be the set of available drivers. We filter these sets such that only orders not yet assigned (orders can only be assigned and accepted once) and drivers with fewer than two active orders are considered. Once we have our viable sets, we aim to minimize the cost function (objective function) where $c_{ij}$ is the cost associated with assigning the $ith$ order to the $jth$ driver. $x_{ij}$ is the decision variable in our program, which is a binary variable of whether we assign the $ith$ order to the $jth$ driver. The cost associated with each order is determine by the last line of @eq-int-1, where we define the cost as a weighted sum of different predictors. In our scenario with Food-Ez, we define four key variables below. 

1. Route Time (total time duration of driver arriving to restaurant, picking up the order, and dropping off the order)
2. Wait Time (total prep time of the order)
3. Drive Acceptance (probability of a driving accepting an order)
4. Driver Idleness (binary of whether or not driver has any orders or no)

This new approach allows us to determine which of our four factors are most pressing for our current business needs since we can adjust the weights of these accordingly and allow the objective function to recognize those changes. For the complete code up of our algorithm, see the [github repo](https://github.com/thebayesianbandit/thebayesianbandit.github.io/blob/main/posts/dispatch_alg/dispatch_alg.ipynb).

In [2]:
#Define order class
class Order:
    def __init__(self, order_id, placement_time, pickup_loc, dropoff_loc, prep_time):
        self.order_id = order_id
        self.placement_time = placement_time
        self.pickup_loc = pickup_loc
        self.dropoff_loc = dropoff_loc
        self.status = "pending"
        self.prep_time = prep_time
        
        self.assigned_driver = None
        self.assigned_time = None
        self.pickup_time = None
        self.dropoff_time = None

In [3]:
#Define driver class
class Driver:
    def __init__(self, driver_id, current_loc, speed=2):
        self.driver_id = driver_id
        self.current_loc = current_loc
        self.status = "idle"
        self.speed = speed
        
        self.assigned_order = []
        self.accept_prob = 0.0
        self.last_order_time = 0.0
        self.total_idle_time = 0.0
        self.total_wait_time = 0.0

In [4]:
#Define restaurant class
class Restaurant:
    def __init__(self, rest_id, loc):
        self.rest_id = rest_id
        self.loc = loc

In [5]:
#Define simulation class
class Simulation:
    def __init__(self, num_drivers, num_rest, grid_size, 
                 order_freq, travel_time_w, wait_time_w, 
                 accept_w, idle_driver_w):
        self.num_drivers = num_drivers
        self.num_rest = num_rest
        self.grid_size = grid_size
        self.order_freq = order_freq
        
        self.travel_time_w = travel_time_w
        self.wait_time_w = wait_time_w
        self.accept_w = accept_w
        self.idle_driver_w = idle_driver_w
        
        self.current_time = 0.0
        self.order_id_counter = 0
        self.rests = {}
        self.orders = {}
        self.drivers = {}
        
    #Calculate distance using euclidean distance
    def calc_dist(self, loc_1, loc_2):
        return np.sqrt((loc_1[0] - loc_2[0])**2 + (loc_1[1] - loc_2[1])**2)
        
    #Generate orders
    def gen_order(self):
        num_orders = np.random.poisson(self.order_freq)
        
        for i in range(num_orders):
            placement_time = self.current_time
            rest = np.random.choice(list(self.rests.keys()))
            pickup_loc = self.rests[rest].loc
            dropoff_loc = (np.random.uniform(0, self.grid_size), np.random.uniform(0, self.grid_size))
            prep_time = np.random.gamma(3, 2)
            
            order = Order(self.order_id_counter, placement_time, pickup_loc, dropoff_loc, prep_time)
            self.orders[order.order_id] = order
            self.order_id_counter += 1
          
    #Generate drivers
    def gen_driver(self):
        for i in range(self.num_drivers):
            driver_id = i
            current_loc = (np.random.uniform(0, self.grid_size), np.random.uniform(0, self.grid_size))
            speed = max(1, np.random.normal(3, 1))
            
            driver = Driver(driver_id, current_loc, speed)
            self.drivers[driver.driver_id] = driver
            
    #Generate restaurants
    def gen_rest(self):
        for i in range(self.num_rest):
            rest_id = i
            loc = (np.random.uniform(0, self.grid_size), np.random.uniform(0, self.grid_size))
            
            rest = Restaurant(rest_id, loc)
            self.rests[rest.rest_id] = rest
            
    #Normalize features
    def get_scores(self, feat_dict):
        temp_lst = []
        scaled_dict = {}
        
        for o_id, d in feat_dict.items():
            scaler = MinMaxScaler()
            scaled_feat = scaler.fit_transform(np.array(list(d.values())).reshape(-1, 1))
            scaled_feat = 1 - scaled_feat
            
            scaled_dict[o_id] = {}
            counter = 0
            for d_id in d.keys():
                scaled_dict[o_id][d_id] = scaled_feat[counter][0]
                counter += 1
        
        return scaled_dict
    
    #Weight normalized features by pre-determined weights
    def weight_scores(self, feat_dict, w):
        for o_id, d in feat_dict.items():
            for d_id, v in d.items():
                feat_dict[o_id][d_id] = v * w
        
        return feat_dict
    
    #Combine feature scores across dictionaries
    def comb_scores(self, lst_dict):
        agg_data = lst_dict[0].copy()
        
        for i in lst_dict:
            for o_id, d in i.items():
                for d_id, v in d.items():
                    agg_data[o_id][d_id] += v
        
        return agg_data
                 
    #Perform distpatch operations
    def dispatch(self):
        orders_pending = [order_id for order_id, order in self.orders.items() if order.status == "pending"]
        drivers_avail = [driver_id for driver_id, driver in self.drivers.items() if len(driver.assigned_order) < 2]
        
        assigned_orders = {}
        pred_eta_dict = {}
        pred_it_dict = {}
        pred_accept_dict = {}
        idle_drivers_dict = {}
        
        if len(drivers_avail) == 0:
            #print(f"No available drivers")
            return assigned_orders
        
        for order_id in orders_pending:
            current_order = self.orders[order_id]
            pred_eta_dict[order_id] = {}
            pred_it_dict[order_id] = {}
            pred_accept_dict[order_id] = {}
            idle_drivers_dict[order_id] = {}
            
            for driver_id in drivers_avail:
                current_driver = self.drivers[driver_id]
                
                dist_to_pickup = self.calc_dist(current_order.pickup_loc, current_driver.current_loc)
                dist_to_dropoff = self.calc_dist(current_order.pickup_loc, current_order.dropoff_loc)
                time_to_pickup = max(2, (dist_to_pickup / current_driver.speed) + np.random.normal(1, .2))
                time_to_dropoff = max(2, (dist_to_dropoff / current_driver.speed) + np.random.normal(1, .2))
                pred_eta = max(self.current_time + time_to_pickup, self.current_time + current_order.prep_time) + time_to_dropoff
                pred_eta_dict[order_id][driver_id] = pred_eta
                
                idle_time = max(0, current_order.prep_time - time_to_pickup)
                pred_it_dict[order_id][driver_id] = idle_time
                
                pred_accept_dict[order_id][driver_id] = -np.random.beta(5,2)
                current_driver.accept_prob = pred_accept_dict[order_id][driver_id] * -1
                
                idle_drivers_dict[order_id][driver_id] = -np.where(len(current_driver.assigned_order) == 0, 1, 0).item()
        
        pred_eta_dict = self.get_scores(pred_eta_dict)
        pred_it_dict = self.get_scores(pred_it_dict)
        
        pred_eta_dict = self.weight_scores(pred_eta_dict, self.travel_time_w)
        pred_it_dict = self.weight_scores(pred_it_dict, self.wait_time_w)
        pred_accept_dict = self.weight_scores(pred_accept_dict, self.accept_w)
        idle_drivers_dict = self.weight_scores(idle_drivers_dict, self.idle_driver_w)
        
        scores_dict = self.comb_scores([pred_eta_dict, pred_it_dict, pred_accept_dict, idle_drivers_dict])
        
        prob = pulp.LpProblem("Dispatch_Problem", pulp.LpMinimize)
        x = pulp.LpVariable.dicts("x", ((i, j) for i in orders_pending for j in drivers_avail), 0, 1, pulp.LpBinary)
        x = {i: {j: x[(i, j)] for j in drivers_avail} for i in orders_pending}
        prob += pulp.lpSum(scores_dict[i][j] * x[i][j] for i in orders_pending for j in drivers_avail), "total_sum"
        
        for i in orders_pending:
            prob += pulp.lpSum(x[i][j] for j in drivers_avail) == 1, f"order_{i}_assignment"
        for j in drivers_avail:
            prob += pulp.lpSum(x[i][j] for i in orders_pending) <= 2, f"max_{j}_orders"
            
        prob.solve(pulp.PULP_CBC_CMD(msg=0))
        
        if prob.status == pulp.LpStatusOptimal:
            #print(f"Optimal solution found for time {self.current_time}")
            for i in orders_pending:
                for j in drivers_avail:
                    if x[i][j].varValue == 1:
                        assigned_orders[i] = j
        else:
            #print(f"No optimal solution found")
            pass
        
        return assigned_orders
    
    #Update driver assignments
    def update_assign(self, assign_dict):
        for o_id, d_id in assign_dict.items():
            current_driver = self.drivers[d_id]
            current_order = self.orders[o_id]
            
            if current_driver.accept_prob > np.random.random():
                current_order.status = "assigned"
                current_order.assigned_driver = d_id
                current_order.assigned_time = self.current_time
                
                current_driver.status = "assigned"
                current_driver.assigned_order.append(o_id)
            else:
                pass
    
    #Update system states
    def update_state(self):
        orders_assign = [order_id for order_id, order in self.orders.items() if order.status == "assigned"]
        orders_route = [order_id for order_id, order in self.orders.items() if order.status == "en route"]
        drivers_idle = [driver_id for driver_id, driver in self.drivers.items() if driver.status == "idle"]
        
        for order_id in orders_assign:
            current_order = self.orders[order_id]
            current_driver = self.drivers[current_order.assigned_driver]
            
            current_order.prep_time = max(0, current_order.prep_time - 1)
            
            dx = current_order.pickup_loc[0] - current_driver.current_loc[0]
            dy = current_order.pickup_loc[1] - current_driver.current_loc[1]
            
            distance = self.calc_dist(current_order.pickup_loc, current_driver.current_loc)
            
            if order_id != current_driver.assigned_order[0]:
                pass
            
            elif distance <= current_driver.speed and current_order.prep_time == 0:
                current_driver.current_loc = current_order.pickup_loc
                current_driver.status = "en route"
                
                current_order.pickup_time = self.current_time
                current_order.status = "en route"
            elif distance <= current_driver.speed:
                current_driver.current_loc = current_order.pickup_loc
                current_driver.total_wait_time += 1
            else:
                dx /= distance
                dy /= distance
                
                current_driver.current_loc = (current_driver.current_loc[0] + (dx * current_driver.speed), 
                                              current_driver.current_loc[1] + (dy * current_driver.speed))
            
        for order_id in orders_route:
            current_order = self.orders[order_id]
            current_driver = self.drivers[current_order.assigned_driver]
           
            dx = current_order.dropoff_loc[0] - current_driver.current_loc[0]
            dy = current_order.dropoff_loc[1] - current_driver.current_loc[1]
            
            distance = self.calc_dist(current_order.dropoff_loc, current_driver.current_loc)
            
            if distance <= current_driver.speed and len(current_driver.assigned_order) < 2:
                current_driver.current_loc = current_order.dropoff_loc
                current_driver.status = "idle"
                current_driver.last_order_time = self.current_time
                current_driver.assigned_order.pop(0)
                
                current_order.status = "delivered"
                current_order.dropoff_time = self.current_time
            elif distance <= current_driver.speed:
                current_driver.current_loc = current_order.dropoff_loc
                current_driver.status = "assigned"
                current_driver.last_order_time = self.current_time
                current_driver.assigned_order.pop(0)
                
                current_order.status = "delivered"
                current_order.dropoff_time = self.current_time
            else:
                dx /= distance
                dy /= distance
                
                current_driver.current_loc = (current_driver.current_loc[0] + (dx * current_driver.speed), 
                                              current_driver.current_loc[1] + (dy * current_driver.speed))
        
        for driver_id in drivers_idle:
            current_driver = self.drivers[driver_id]
            current_driver.total_idle_time += 1
            
    def run(self, sim_duration):
        print(f"Starting simulation for {sim_duration} minutes")
        np.random.seed(42)
        self.gen_driver()
        self.gen_rest()
        
        while self.current_time < sim_duration:
            np.random.seed(42)
            self.gen_order()
            dispatch_dict = self.dispatch()
            self.update_assign(dispatch_dict)
            self.update_state()
            # if self.current_time % 10 == 0:
            #     for i, j in self.orders.items():
            #         print(f"Order {i} status: {j.status}")
            #     print("\n")
            self.current_time += 1
            

# Simulation Results

We have successfully defined a new approach for our Food-Ez dispatch algorithm. To assess its efficacy in real-world scenarios, we designed a simulation to test it. We ran 5 simulations for 120 iterations (120 simulation "minutes"). The simulation parameters consist of the following: number of drivers (which we hold constant throughout), number of restaurants, grid size of city, order arrival rate, route time weight, wait time weight, driver acceptance weight, and driver idleness weight. Each simulation with their respective parameters are found below.

1. Simulation(10, 10, 5, 1, .5, .2, .1, .2)
2. Simulation(5, 10, 5, 1, .5, .2, .1, .2)
3. Simulation(10, 10, 5, 1, .05, .7, .05, .2)
4. Simulation(10, 10, 5, 1, .25, .25, .25, .25)
5. Simulation(10, 10, 5, 1, .8, .05, .1, .05)

For example, simulation 1 has 10 drivers with 10 restaurants, a city that is 5x5 units, an order arrival rate of 1 (per minute), .5 weight on route time, .2 weight on wait time, .1 weight on driver acceptance, and .2 weight on driver idleness. The results of each simulation is found below in @tbl-res-1. (Note: we ran each simulation with the same random seed settings for consistency in comparison).

In [6]:
#Define simulations
sim_1 = Simulation(10, 10, 5, 1, .5, .2, .1, .2)
sim_2 = Simulation(5, 10, 5, 1, .5, .2, .1, .2)
sim_3 = Simulation(10, 10, 5, 1, .05, .7, .05, .2)
sim_4 = Simulation(10, 10, 5, 1, .25, .25, .25, .25)
sim_5 = Simulation(10, 10, 5, 1, .8, .05, .1, .05)

In [7]:
#| output: false

#Run simulations
sim_1.run(120)
sim_2.run(120)
sim_3.run(120)
sim_4.run(120)
sim_5.run(120)

Starting simulation for 120 minutes
Starting simulation for 120 minutes
Starting simulation for 120 minutes
Starting simulation for 120 minutes
Starting simulation for 120 minutes


In [8]:
#Define function for reporting simulation statistics
def get_sum_stats(sim_object):
    status_counts = {
        "# orders": 0,
        "pending": 0,
        "assigned": 0,
        "en route": 0,
        "delivered": 0
    }

    for i, j in sim_object.orders.items():
        status_counts[j.status] += 1
        status_counts["# orders"] += 1
    print(f"Order Status Summary: {status_counts}")
    
    total_wait_time = 0
    total_idle_time = 0
    
    for i, j in sim_object.drivers.items():
        total_wait_time += j.total_wait_time
        total_idle_time += j.total_idle_time
    print(f"Average Driver Wait Time: {total_wait_time / len(sim_object.drivers.keys()):.2f}")
    print(f"Average Driver Idle Time: {total_idle_time / len(sim_object.drivers.keys()):.2f}")
    
    total_time_pick = 0
    total_time_drop = 0
    total_time_route = 0
    total_time_pending = 0
    delivered_ids = []
    en_route_ids = []
    assigned_ids = []

    for i, j in sim_object.orders.items():
        if j.status == "delivered":
            total_time_pick += (j.pickup_time - j.assigned_time)
            total_time_drop += (j.dropoff_time - j.pickup_time)
            total_time_route += (j.dropoff_time - j.assigned_time)
            total_time_pending += (j.assigned_time - j.placement_time)
            
            delivered_ids.append(i)
            en_route_ids.append(i)
            assigned_ids.append(i)
        elif j.status == "en route":
            total_time_pick += (j.pickup_time - j.assigned_time)
            total_time_pending += (j.assigned_time - j.placement_time)
            
            en_route_ids.append(i)
            assigned_ids.append(i)
        elif j.status == "assigned":
            total_time_pending += (j.assigned_time - j.placement_time)
            assigned_ids.append(i)
    print(f"Average Time To Pickup: {total_time_pick / len(en_route_ids):.2f}")
    print(f"Average Time To Dropoff: {total_time_drop / len(delivered_ids):.2f}")
    print(f"Average Time Of Total Route: {total_time_route / len(delivered_ids):.2f}")
    print(f"Average Time Pending: {total_time_pending / len(assigned_ids):.2f}")

In [9]:
#| output: false

#Show stats for sim 1
get_sum_stats(sim_1)

Order Status Summary: {'# orders': 120, 'pending': 0, 'assigned': 7, 'en route': 1, 'delivered': 112}
Average Driver Wait Time: 21.50
Average Driver Idle Time: 59.50
Average Time To Pickup: 6.78
Average Time To Dropoff: 1.73
Average Time Of Total Route: 8.52
Average Time Pending: 0.40


In [10]:
#| output: false

#Show stats for sim 2
get_sum_stats(sim_2)

Order Status Summary: {'# orders': 120, 'pending': 0, 'assigned': 6, 'en route': 1, 'delivered': 113}
Average Driver Wait Time: 71.40
Average Driver Idle Time: 2.80
Average Time To Pickup: 6.00
Average Time To Dropoff: 1.00
Average Time Of Total Route: 7.00
Average Time Pending: 0.00


In [11]:
#| output: false

#Show stats for sim 3
get_sum_stats(sim_3)

Order Status Summary: {'# orders': 120, 'pending': 0, 'assigned': 7, 'en route': 1, 'delivered': 112}
Average Driver Wait Time: 53.20
Average Driver Idle Time: 41.30
Average Time To Pickup: 6.24
Average Time To Dropoff: 1.13
Average Time Of Total Route: 7.38
Average Time Pending: 0.42


In [12]:
#| output: false

#Show stats for sim 4
get_sum_stats(sim_4)

Order Status Summary: {'# orders': 120, 'pending': 0, 'assigned': 7, 'en route': 1, 'delivered': 112}
Average Driver Wait Time: 31.90
Average Driver Idle Time: 51.00
Average Time To Pickup: 6.32
Average Time To Dropoff: 1.64
Average Time Of Total Route: 7.96
Average Time Pending: 0.25


In [13]:
#| output: false

#Show stats for sim 5
get_sum_stats(sim_5)

Order Status Summary: {'# orders': 120, 'pending': 1, 'assigned': 6, 'en route': 2, 'delivered': 111}
Average Driver Wait Time: 19.40
Average Driver Idle Time: 61.00
Average Time To Pickup: 6.73
Average Time To Dropoff: 1.76
Average Time Of Total Route: 8.50
Average Time Pending: 0.32


| Simulation | Orders | Pending | Assigned | En Route | Delivered | Avg Driver Wait Time | Avg Driver Idle Time | Avg Time To Pickup | Avg Time To Dropoff | Avg Time Of Total Route | Avg Time Pending |
|:----------:|:------:|:-------:|:--------:|:--------:|:---------:|:--------------------:|:--------------------:|:------------------:|:-------------------:|:-----------------------:|:----------------:|
| 1          | 120    | 0       | 7        | 1        | 112       | 21.50                | 59.50                | 6.78               | 1.73                | 8.52                    | 0.40             |
| 2          | 120    | 0       | 6        | 1        | 113       | 71.40                | 2.80                 | 6.00               | 1.00                | 7.00                    | 0.00             |
| 3          | 120    | 0       | 7        | 1        | 112       | 53.20                | 41.30                | 6.24               | 1.13                | 7.38                    | 0.42             |
| 4          | 120    | 0       | 7        | 1        | 112       | 31.90                | 51.00                | 6.32               | 1.64                | 7.96                    | 0.25             |
| 5          | 120    | 1       | 6        | 2        | 111       | 19.40                | 61.00                | 6.73               | 1.76                | 8.50                    | 0.32             |

: Simulation Results {#tbl-res-1}

From @tbl-res-1, the number of deliveries performed appear to be consistent even with the change of parameter weights. This is a nice observation since simulation 2 had half of the drivers that the other simulations had, but delivered the most orders. Furthermore, simulation 2 shows that drivers were used efficiently as most drivers were only idle for on about 3 minutes on average, compared to the other simulations that had far more idle time. We continue to see this trend in other metrics for simulation 2. Simulation 2 had the lowest average time to dropoff, lowest average time of total route, and even lowest pending time for orders in the queue. Further investigation is needed to see what the optimal driver supply would be in different scenarios, but this is a telling sign at the moment that our new algorithm utilizes driver supply more efficiently.

There are many other interesting findings from these 5 simulations, but we will leave these to the reader to discover. 

# Conclusion

In this post, we presented a business case where a food-delivery company, Food-Ez, is looking to optimize their dispatch algorithm to accomodate more business objectives. We then walked through how this problem can be designed as a control problem and identified the system to be controlled, the feedback data needed to control it, and how to design our controller. 

Our controller in this scenario is the dispatch algorithm itself, designed to more efficiently assign orders by raising the limit on orders a driver can carry along with other business objectives within our objective function. To do this, we changed our algorithm foundation from a bipartite matching problem to an integer programming program. We formulated how we would minimize this new objective function and designed a simulation to demonstrate the usefulness of it.

Next steps would include further simulations with more "real-world" dynamics involved and potential experimentation using things like switchback experiments. Overall, we hope this post demonstrated key frameworks that one can use in designing real-world applications to enhance operational efficiency for businesses.